In [4]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pyspark

In [6]:
# import pandas as pd
# pd.read_csv("data.csv");

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark=SparkSession.builder.appName("Task2").getOrCreate()

23/03/01 19:53:50 WARN Utils: Your hostname, Kemboi resolves to a loopback address: 127.0.1.1; using 10.3.3.191 instead (on interface wlo1)
23/03/01 19:53:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 19:53:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark

In [10]:
df=spark.read.csv("data.csv")

In [11]:
# spark.read.option("header","true").csv("data.csv").show()

In [12]:
from pyspark.sql.functions import col
import matplotlib.pyplot as plt
# Compute summary statistics
summary = df.describe().toPandas()

23/03/01 19:56:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
# Print summary statistics
print(summary)


  summary                 _c0                _c1                  _c2  \
0   count             1048576            1048576              1048576   
1    mean   4876.261774312758  8.107557399327659                 None   
2  stddev  14443.436003519171   4.46062464438205                 None   
3     min                   0                  0  05/01/1970 03:01:17   
4     max            Dst Port           Protocol            Timestamp   

                    _c3                 _c4                 _c5  \
0               1048576             1048576             1048576   
1     6255554.608775719   6.206622320768663   7.211191378775958   
2  1.2602911086328785E9  44.478512313301124  104.86818042875132   
3          -11873000000                   1                   0   
4         Flow Duration        Tot Fwd Pkts        Tot Bwd Pkts   

                  _c6                _c7                 _c8  ...  \
0             1048576            1048576             1048576  ...   
1  447.9936428009441

In [14]:
# Show the names of all columns in a DataFrame
print(df.columns)


['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40', '_c41', '_c42', '_c43', '_c44', '_c45', '_c46', '_c47', '_c48', '_c49', '_c50', '_c51', '_c52', '_c53', '_c54', '_c55', '_c56', '_c57', '_c58', '_c59', '_c60', '_c61', '_c62', '_c63', '_c64', '_c65', '_c66', '_c67', '_c68', '_c69', '_c70', '_c71', '_c72', '_c73', '_c74', '_c75', '_c76', '_c77', '_c78', '_c79']


In [15]:
summary2 = df.describe()
summary2.show()

+-------+------------------+-----------------+-------------------+--------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+-------+-------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+-------------------+-------------+-------------+-------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+--------------------+-------------------+--------------------+-------------------+------------------+-------------------+--------------+-------------------

In [ ]:
from pyspark.sql.functions import col

df = df.withColumn('_c3_float', col('_c3').cast('float'))
df = df.withColumn('_c0_float', col('_c0').cast('float'))

corr_value = df.corr('_c0_float', '_c3_float')
print("Correlation value between column1 and column4 is:", corr_value)



In [17]:
# Encode the categorical feature using OneHotEncoder
encoder = OneHotEncoder(inputCols=['indexed_c0'], outputCols=['c0_encoded'])
encoded_df = encoder.fit(indexed_df).transform(indexed_df)

# Assemble the feature columns into a single vector column
assembler = VectorAssembler(inputCols=['c0_encoded', '_c3'], outputCol='features')
hypothesis_test_df = assembler.transform(encoded_df)

# Create a ChiSquareTest object
chisqTest = ChiSquareTest.test(hypothesis_test_df, 'features', '_c4')

# Show the result of the hypothesis test
chisqTest.show()



NameError: name 'OneHotEncoder' is not defined

In [47]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
import numpy as np
import matplotlib.pyplot as plt

# Load data into PySpark DataFrame
data = spark.read.format("csv").option("header", "true").load("data.csv")

# Extract a single column as data to estimate density for
column_name = "Flow Duration"
column = data.select(col(column_name))

# Define a user-defined function (UDF) to calculate Gaussian kernel density
def gaussian_kernel_density(x, column, h):
    kernel = np.exp(-(column - x)**2 / (2 * h**2)) / np.sqrt(2 * np.pi * h**2)
    density = np.mean(kernel)
    return density

gaussian_kernel_density_udf = udf(lambda x: float(gaussian_kernel_density(x, column, h=0.1)), DoubleType())

# Generate range of values to evaluate density at
x_vals = np.linspace(column.agg({"Flow Duration": "min"}).collect()[0][0], 
                     column.agg({"Flow Duration": "max"}).collect()[0][0], 
                     num=1000)


# Evaluate density at the generated values
y_vals = np.array([gaussian_kernel_density_udf(x).item() for x in x_vals])

# Plot density estimate
plt.plot(x_vals, y_vals)
plt.title('Density Estimate')
plt.xlabel('Values')
plt.ylabel('Density')
plt.show()


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [43]:
# Random Forest

from pyspark.sql.functions import col, when
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load the data
data = spark.read.csv("data.csv", header=True, inferSchema=True)

# Filter out rows with missing values
data = data.dropna()

# Map the labels to 0 or 1 (attack or normal)
data = data.withColumn("Label", when(col("Label") == "BENIGN", 0).otherwise(1))

# Select only the required columns
selected_cols = ["Dst Port", "Protocol", "Tot Fwd Pkts", "Tot Bwd Pkts", "Pkt Len Mean"]
data = data.select(selected_cols + ["Label"])

# Assemble the features into a single vector column
assembler = VectorAssembler(inputCols=selected_cols, outputCol="features")
data = assembler.transform(data)

# Split the data into training and testing sets
(training_data, testing_data) = data.randomSplit([0.7, 0.3], seed=42)

# Train the decision tree classifier
dt = DecisionTreeClassifier(labelCol="Label", featuresCol="features", maxDepth=5)
dt_model = dt.fit(training_data)

# Make predictions on the testing data
predictions = dt_model.transform(testing_data)

# Evaluate the model using the accuracy metric
evaluator = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Accuracy: {accuracy}")


Accuracy: 1.0


In [45]:
# TASK 1
# Big Data Query & Analysis using Spark SQL [30 marks]


from pyspark.sql import SparkSession

# create a SparkSession object
spark = SparkSession.builder.appName("my_app").getOrCreate()

# load your data into a DataFrame
df = spark.read.format("csv").option("header", True).load("data.csv")

# create a temporary view of the DataFrame
df.createOrReplaceTempView("my_table")

# run a SQL query against the view
sql = "SELECT `Dst Port`, (COUNT(CASE WHEN `FIN Flag Cnt` >= 2 THEN 1 END) / COUNT(*) * 100) AS FIN_Flag_Percentage FROM my_table GROUP BY `Dst Port` ORDER BY FIN_Flag_Percentage DESC"
result = spark.sql(sql)

# display the result
result.show()

+--------+-------------------+
|Dst Port|FIN_Flag_Percentage|
+--------+-------------------+
|   57868|                0.0|
|   35640|                0.0|
|   51946|                0.0|
|   37796|                0.0|
|   43044|                0.0|
|   34622|                0.0|
|   57048|                0.0|
|   54580|                0.0|
|   52996|                0.0|
|   37806|                0.0|
|   40090|                0.0|
|   35792|                0.0|
|   46664|                0.0|
|   60042|                0.0|
|   37128|                0.0|
|   44656|                0.0|
|   49978|                0.0|
|   50214|                0.0|
|   53624|                0.0|
|   56400|                0.0|
+--------+-------------------+
only showing top 20 rows



In [46]:
from pyspark.sql import SparkSession

# create a SparkSession object
spark = SparkSession.builder.appName("my_app").getOrCreate()

# load your data into a DataFrame
df = spark.read.format("csv").option("header", True).load("data.csv")

# create a temporary view of the DataFrame
df.createOrReplaceTempView("my_table")

# run a SQL query against the view
sql = "SELECT Protocol, AVG(`Flow Duration`) AS Avg_Flow_Duration FROM my_table GROUP BY Protocol ORDER BY Avg_Flow_Duration DESC LIMIT 5"
result = spark.sql(sql)

# display the result
result.show()

+--------+--------------------+
|Protocol|   Avg_Flow_Duration|
+--------+--------------------+
|       6|   9549803.667116841|
|      17|  376613.15135208116|
|       0|-1.21059038651405...|
+--------+--------------------+

